In [ ]:
pwd

In [ ]:
# Copiar el archivo oauth.py en el directorio que se muestra en la ejecución anterior
%run "oauth.py"

In [ ]:
# Aquí introducimos nuestras claves de Twitter
def oauth() :
    return { "consumer_key" : "XXXX",
        "consumer_secret" : "XXXX",
        "token_key" : "XXXX",
        "token_secret" : "XXXX" }

In [ ]:
# Esta función va a generar los parámetros necesarios para generar la URL
def augment(url, parameters) :
    secrets = oauth()
    consumer = OAuthConsumer(secrets['consumer_key'], secrets['consumer_secret'])
    token = OAuthToken(secrets['token_key'],secrets['token_secret'])

    oauth_request = OAuthRequest.from_consumer_and_token(consumer, 
        token=token, http_method='GET', http_url=url, parameters=parameters)
    oauth_request.sign_request(OAuthSignatureMethod_HMAC_SHA1(), consumer, token)
    return oauth_request.to_url()

In [ ]:
import urllib
import json
TWITTER_URL = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

# Cuenta de Twitter de la que vamos a extraer los últimos 2 tweets
acct = 'eoi'

# Cantidad de tweets que deseamos extraer
n_tweets = 2

# Creamos la URL
url = augment(TWITTER_URL,
    {'screen_name': acct, 'count': n_tweets,'include_rts':'True','tweet_mode':'extended'} )
print(url)

# Nos conectamos a la URL anterior y extraemos el json que nos devuelve la API 
connection = urllib.request.urlopen(url)

# Datos de línea de tiempo de la cuenta seleccionada
data = connection.read().decode()
js = json.loads(data)

# Imprimimos el json
#print(json.dumps(js,indent=4))

# Además de los datos, nos devuelve una cabecera de respuesta HTTP
headers = dict(connection.getheaders())
# De la cabecera podemos extraer las peticiones restantes que tenemos
print('Remaining', headers['x-rate-limit-remaining'])

# Por último imprimimos los tweets
for k in range(0,n_tweets):
    tweet = js[k]["full_text"]
    print('\nTweet ' + str(k+1) + ': '+ tweet)

Llamamos a otra API para obtener la lista de amigos de un usuario

In [ ]:
import urllib.request, urllib.parse, urllib.error
import json


TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'


while True:
    print('')
    acct = input('Enter Twitter Account:')
    if (len(acct) < 1): break
    url = augment(TWITTER_URL,
                        {'screen_name': acct, 'count': '5'})
    #print('Retrieving', url)
    connection = urllib.request.urlopen(url)
    data = connection.read().decode()

    js = json.loads(data)
    print(json.dumps(js, indent=2))

    headers = dict(connection.getheaders())
    print('Remaining', headers['x-rate-limit-remaining'])

    for u in js['users']:
        print(u['screen_name'])